# Detector de imagenes (Fumadores)

## Importación del dataset

In [1]:
# Código

## Entrenamiento de modelos

### SVC - Sklearn

In [5]:
# Importaciones necesarias

#### Tratamiento de datos

In [2]:
# Código

#### Entrenamiento del modelo svc

In [3]:
# Código

#### Exportación de recursos (modelo, scaler, etc)

In [4]:
# Código